In [ ]:
import torch
import numpy as np

import time
import matplotlib.pyplot as plt

from utils.SineDataSet import get_batch
from utils.Generator import Generator

# Initialize generators and optimizers

In [ ]:
G = Generator().cuda()
optimizer_G = torch.optim.Adam(G.parameters(), lr=1e-4)

# Training & logging

In [ ]:
mean_errors_to_groundtruth = []
ticks = []
losses_G = []
tick = time.time()

In [ ]:
while True:
    with torch.no_grad():
        y, yδ, _ = get_batch()

    optimizer_G.zero_grad()
    y_approximation = G(yδ)
    loss = torch.mean((y - y_approximation)**2)
    losses_G.append(loss.item())
    loss.backward()
    optimizer_G.step()

    with torch.no_grad():
        mean_errors_to_groundtruth.append(torch.norm(y - y_approximation, dim=1).mean().item())
        ticks.append(time.time() - tick)

    if not len(mean_errors_to_groundtruth) % 100:
        torch.save(G.state_dict(), f'trained_models/supervised/G_state_{len(mean_errors_to_groundtruth)}.pt')
        np.save(f'trained_models/supervised/mean_errors_to_groundtruth.npy', mean_errors_to_groundtruth)
        np.save(f'trained_models/supervised/ticks.npy', ticks)
        np.save(f'trained_models/supervised/losses_G.npy', losses_G)